<a href="https://colab.research.google.com/github/priyarajmohan/ML_projects/blob/main/AI_Based_Medical_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
from datetime import datetime

# ==============================
# Medical Knowledge Base (Rules)
# ==============================

RULES = [
    {
        "name": "Viral Fever",
        "conditions": {
            "fever_days": lambda x: x >= 3,
            "body_pain": True,
            "fatigue": True
        },
        "advice": "Possible viral infection. Please consult a doctor.",
        "explanation": "Fever for more than 3 days with body pain and fatigue indicates viral infection."
    },
    {
        "name": "Common Cold",
        "conditions": {
            "runny_nose": True,
            "sore_throat": True,
            "fever": False
        },
        "advice": "You may have a common cold. Take rest and drink warm fluids.",
        "explanation": "Runny nose and sore throat without fever usually indicate a common cold."
    },
    {
        "name": "Food Poisoning",
        "conditions": {
            "vomiting": True,
            "diarrhea": True
        },
        "advice": "Possible food poisoning. Stay hydrated and consult a doctor if symptoms persist.",
        "explanation": "Vomiting and diarrhea together indicate digestive infection."
    },
    {
        "name": "Emergency Case",
        "conditions": {
            "chest_pain": True,
            "breathing_difficulty": True
        },
        "advice": "🚨 Emergency! Seek immediate medical help.",
        "explanation": "Chest pain and breathing difficulty can indicate serious medical conditions."
    }
]

# ==============================
# Session State for History
# ==============================

if "history" not in st.session_state:
    st.session_state.history = []

# ==============================
# Inference Engine
# ==============================

def match_rule(rule, user_data):
    for key, condition in rule["conditions"].items():
        user_value = user_data.get(key)

        if callable(condition):
            if not condition(user_value):
                return False
        else:
            if user_value != condition:
                return False
    return True


def infer(user_data):
    matched = []
    for rule in RULES:
        if match_rule(rule, user_data):
            matched.append(rule)
    return matched


def generate_output(matched_rules):
    if not matched_rules:
        return (
            "No matching condition found. Please consult a doctor.",
            "Symptoms did not match any predefined rules."
        )

    advice = []
    explanation = []

    for rule in matched_rules:
        advice.append(f"✅ **{rule['name']}**: {rule['advice']}")
        explanation.append(f"🔍 {rule['explanation']}")

    return "\n\n".join(advice), "\n\n".join(explanation)


# ==============================
# Streamlit UI
# ==============================

st.set_page_config(page_title="AI Medical Assistant", layout="centered")
st.title("🩺 AI-Based Medical Assistant")
st.write("Rule-Based Expert System for Preliminary Medical Guidance")

st.divider()

# Input Section
st.subheader("📝 Enter Symptoms")

col1, col2 = st.columns(2)

with col1:
    fever = st.checkbox("Fever")
    body_pain = st.checkbox("Body Pain")
    fatigue = st.checkbox("Fatigue")
    runny_nose = st.checkbox("Runny Nose")
    sore_throat = st.checkbox("Sore Throat")

with col2:
    vomiting = st.checkbox("Vomiting")
    diarrhea = st.checkbox("Diarrhea")
    chest_pain = st.checkbox("Chest Pain")
    breathing_difficulty = st.checkbox("Breathing Difficulty")
    fever_days = st.number_input("Days of Fever", min_value=0, max_value=30)

user_data = {
    "fever": fever,
    "fever_days": fever_days,
    "body_pain": body_pain,
    "fatigue": fatigue,
    "runny_nose": runny_nose,
    "sore_throat": sore_throat,
    "vomiting": vomiting,
    "diarrhea": diarrhea,
    "chest_pain": chest_pain,
    "breathing_difficulty": breathing_difficulty
}

# Button
if st.button("🔍 Analyze Symptoms"):
    matched_rules = infer(user_data)
    advice, explanation = generate_output(matched_rules)

    st.success("✅ Analysis Complete")

    st.subheader("💡 Medical Advice")
    st.markdown(advice)

    st.subheader("📖 Explanation")
    st.markdown(explanation)

    # Save History
    st.session_state.history.append({
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "data": user_data,
        "advice": advice
    })

st.divider()

# History Section
st.subheader("📂 Advisory History")

if st.session_state.history:
    for i, record in enumerate(reversed(st.session_state.history), 1):
        with st.expander(f"Record {i} — {record['time']}"):
            st.write("**Symptoms:**", record["data"])
            st.write("**Advice:**")
            st.markdown(record["advice"])
else:
    st.info("No history available yet.")

st.caption("⚠ This system provides only preliminary guidance and does not replace professional medical advice.")